<a href="https://colab.research.google.com/github/ajaykumarmehra/Toxic-Comment-Detection-and-Classification/blob/main/Support_Vector_Machine(Binary_Relevance_and_ClassifierChain)_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path='/content/gdrive/My Drive/dataset_toxic_comment/'
import pandas as pd
train=pd.read_csv(path+"train.csv")
test=pd.read_csv(path+"test.csv")
labels = pd.read_csv(path + "test_labels.csv")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
classes = list(train.columns)[2:]
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
train[classes].sum()

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [ ]:
links = '(http://.*?\s)|(http://.*)'
ip_addr = '\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
users = '\[\[User.*'
newline = '\\n'
def clean(comment):
  import re
  comment=comment.lower()
  comment=re.sub(links,'',comment)
  comment=re.sub(ip_addr,'',comment)
  comment=re.sub(users,'',comment)
  comment=re.sub(newline,'',comment)
  return comment

In [ ]:
train['comment_text']=train['comment_text'].map(lambda i:clean(i))
test['comment_text']=test['comment_text'].map(lambda i:clean(i))

In [ ]:
x=train['comment_text']
y=train.iloc[:,2:8]
print(x.shape)
print(y.shape)

(159571,)
(159571, 6)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
labels = pd.read_csv(path + "test_labels.csv")
labels=labels.iloc[:,1:]
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [ ]:
labels_consider=labels[idx]
test=test.iloc[:,1:]
tests_consider=test[idx].values[:,0]

print(labels_consider.shape, tests_consider.shape)

(63978, 6) (63978,)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000,strip_accents='unicode',stop_words='english',token_pattern=r'\w{2,}')

In [ ]:
tfidf.fit(X_train)
X_train_feat=tfidf.fit_transform(X_train)
print(X_train_feat.shape)

X_val_feat=tfidf.transform(X_val)
print(X_val_feat.shape)

X_test_feat=tfidf.transform(tests_consider)
print(X_test_feat.shape)

(127656, 5000)
(31915, 5000)
(63978, 5000)


In [ ]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 3.9MB/s 


In [ ]:
print(X_train.shape)
print(y_train.shape)

(127656,)
(127656, 6)


In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

model_br = BinaryRelevance(classifier = SVC(), require_dense = [False, True])
model_br.fit(X_train_feat, y_train)

BinaryRelevance(classifier=SVC(C=1.0, break_ties=False, cache_size=200,
                               class_weight=None, coef0=0.0,
                               decision_function_shape='ovr', degree=3,
                               gamma='scale', kernel='rbf', max_iter=-1,
                               probability=False, random_state=None,
                               shrinking=True, tol=0.001, verbose=False),
                require_dense=[False, True])

In [ ]:
preds_train = model_br.predict(X_train_feat)

In [ ]:
from sklearn.metrics import roc_auc_score, hamming_loss, precision_score, recall_score, f1_score,

print(roc_auc_score(y_train, preds_train.toarray()))


0.7849777264669738


In [ ]:
preds_val = model_br.predict(X_val_feat)

In [ ]:
print(roc_auc_score(y_val, preds_val.toarray()))

0.6775881668496974


In [ ]:
preds_test = model_br.predict(X_test_feat)

In [ ]:
from sklearn.metrics import roc_auc_score, hamming_loss, accuracy_score, log_loss
print("ROC AUC Score: ", roc_auc_score(labels_consider, preds_test.toarray()))
print("Hamming Loss: ", hamming_loss(labels_consider, preds_test.toarray()))
print("Log Loss: ", log_loss(labels_consider, preds_test.toarray()))
print("Accuracy Score: ", accuracy_score(labels_consider, preds_test.toarray()))

ROC AUC Score:  0.6886074082694079
Hamming Loss:  0.02672533266643742
Log Loss:  1.6236715870057155
Accuracy Score:  0.8928225327456313


In [ ]:
import pickle
path = '/content/gdrive/My Drive/dataset_toxic_comment/saved_model/SVM_Binary_Relevance/'
pickle.dump(model_br, open(path + "SVM_Binary_Relevance.sav", 'wb'))

In [ ]:
loaded_model = pickle.load(open(path + "SVM_Binary_Relevance.sav", 'rb'))
preds_test = model_br.predict(X_test_feat)
print(roc_auc_score(labels_consider, preds_test.toarray()))

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC

model_cc = ClassifierChain(classifier = SVC(), require_dense = [False, True])
model_cc.fit(X_train_feat, y_train)

ClassifierChain(classifier=SVC(C=1.0, break_ties=False, cache_size=200,
                               class_weight=None, coef0=0.0,
                               decision_function_shape='ovr', degree=3,
                               gamma='scale', kernel='rbf', max_iter=-1,
                               probability=False, random_state=None,
                               shrinking=True, tol=0.001, verbose=False),
                order=None, require_dense=[False, True])

In [ ]:
preds_train = model_cc.predict(X_train_feat)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(roc_auc_score(y_train, preds_train.toarray()))

0.7804921656671743


In [ ]:

preds_val = model_cc.predict(X_val_feat)

In [ ]:
print(roc_auc_score(y_val, preds_val.toarray()))

0.6921115346926116


In [ ]:
preds_test = model_cc.predict(X_test_feat)

In [ ]:
from sklearn.metrics import roc_auc_score, hamming_loss, accuracy_score, log_loss
print("ROC AUC Score: ", roc_auc_score(labels_consider, preds_test.toarray()))
print("Hamming Loss: ", hamming_loss(labels_consider, preds_test.toarray()))
print("Log Loss: ", log_loss(labels_consider, preds_test.toarray()))
print("Accuracy Score: ", accuracy_score(labels_consider, preds_test.toarray()))

ROC AUC Score:  0.7005223066528395
Hamming Loss:  0.02836912688736753
Log Loss:  1.5139978269884409
Accuracy Score:  0.892994466847979


In [ ]:
path = '/content/gdrive/My Drive/dataset_toxic_comment/saved_model/SVM_Classifier_Chain/'
pickle.dump(model_br, open(path + "SVM_Classifier_Chain.sav", 'wb'))